In [6]:
import numpy as np

In [2]:
from coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from image_utils import image_from_url


In [3]:
def show_image(batch_size):
    captions, features, urls = sample_coco_minibatch(data, batch_size=batch_size)
    for i, (caption, url) in enumerate(zip(captions, urls)):
        plt.imshow(image_from_url(url))
        plt.axis('off')
        caption_str = decode_captions(caption, data['idx_to_word'])
        plt.title(caption_str)
        plt.show()


In [4]:
data = load_coco_data(pca_features=False, max_train=20)

In [7]:
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

word_to_idx <class 'dict'> 1004
train_features <class 'numpy.ndarray'> (82783, 4096) float32
val_captions <class 'numpy.ndarray'> (195954, 17) int32
val_urls <class 'numpy.ndarray'> (40504,) <U63
train_urls <class 'numpy.ndarray'> (82783,) <U63
val_image_idxs <class 'numpy.ndarray'> (195954,) int32
val_features <class 'numpy.ndarray'> (40504, 4096) float32
train_captions <class 'numpy.ndarray'> (20, 17) int32
train_image_idxs <class 'numpy.ndarray'> (20,) int32
idx_to_word <class 'list'> 1004


In [ ]:
show_image(3)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import layer_utils 

## word embedding
vocab_dim = len(data['word_to_idx'])
embedding_dim = 300

## map 
sy_caption_input = tf.placeholder(shape=[None, None, vocab_dim], name="ob", dtype=tf.float32)

layer_utils.build_mlp()



x = tf.placeholder(shape=(None,None,10),dtype=tf.float32)
cell = tf.nn.rnn_cell.LSTMCell(30)
outputs, final = tf.nn.dynamic_rnn(cell, x, time_major=False, dtype=tf.float32)


In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

import numpy as np
o, f = sess.run([outputs, final], {x: np.random.rand(32,50,10)})
o.shape
